<a href="https://colab.research.google.com/github/yokosmina/Taller-Ciencia-/blob/main/tallerdia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import pandas as pd
import numpy as np

url='https://raw.githubusercontent.com/b414m/TallerDeCienciaDeDatos/main/bots_and_humans_metrics_dataset_es.csv'
dataset = pd.read_csv(url)
dataset = dataset[['nature','Class','num_links','sum_emojis']].reset_index(drop=True)

miniDataset = dataset.sample(n=10).reset_index(drop=True)
miniDataset



,nature,Class,num_links,sum_emojis
0,human,0,0,22
1,human,0,0,4
2,human,0,0,5
3,human,0,0,4
4,bot,1,1,0
5,bot,1,1,0
6,bot,1,0,5
7,bot,1,0,94
8,human,0,0,0
9,bot,1,1,0


Division del conjunto de datos en Entrenamiento y Prueba

Datos de entrenamiento

---



In [3]:
train = miniDataset.head (n = int((70*miniDataset.shape[0])/100))
train

,nature,Class,num_links,sum_emojis
0,bot,1,1,300
1,human,0,0,0
2,bot,1,1,58
3,human,0,0,1
4,bot,1,0,0
5,human,0,0,37
6,bot,1,1,0


Datos de Prueba

In [4]:
test = miniDataset.tail(n=int((30*miniDataset.shape[0])/100))
test

,nature,Class,num_links,sum_emojis
7,bot,1,1,0
8,human,0,0,17
9,human,0,1,121


variables para el algoritmo Naive Bayes
1.driver. es el nombre de la caracteriica analizada del conjunto de datos


> Indented block



In [5]:
columns = train.columns
drivers = [column for column in columns if (column != 'nature' and column != 'Class')]
drivers


['num_links', 'sum_emojis']

2. etiqueta o label


In [9]:
labels = [column for column in columns if (column == 'nature' or column == 'Class')]
labels


['nature', 'Class']

In [10]:
train

,nature,Class,num_links,sum_emojis
0,bot,1,1,300
1,human,0,0,0
2,bot,1,1,58
3,human,0,0,1
4,bot,1,0,0
5,human,0,0,37
6,bot,1,1,0


In [8]:
Value = train['sum_emojis']
Value

0    300
1      0
2     58
3      1
4      0
5     37
6      0
Name: sum_emojis, dtype: int64

N. es el numero total de registros


In [12]:
N = train.shape[0]
N

7

5. NC: es el númeor de registros relacionados a la clase

In [16]:
train

,nature,Class,num_links,sum_emojis
0,bot,1,1,300
1,human,0,0,0
2,bot,1,1,58
3,human,0,0,1
4,bot,1,0,0
5,human,0,0,37
6,bot,1,1,0


In [15]:
NC = train[train['Class']==1].shape[0]
NC


4

6.NNOC: es el números de registros relacionados a la no clase


In [17]:
NNOC = train[train['Class']==0].shape[0]
NNOC


3

7.NXs: es la frecuencia correspondiente a cada valor diferente de la característica analizada

In [18]:
train

,nature,Class,num_links,sum_emojis
0,bot,1,1,300
1,human,0,0,0
2,bot,1,1,58
3,human,0,0,1
4,bot,1,0,0
5,human,0,0,37
6,bot,1,1,0


In [19]:
NXsLinks = train['num_links'].value_counts().reset_index(name='NXsLinks')
NXsLinks = NXsLinks.rename(columns={'index':'num_links'})
NXsLinks


,num_links,NXsLinks
0,0,4
1,1,3


In [20]:
NXsSumEmojis = train['sum_emojis'].value_counts().reset_index(name='NXsSumEmojis')
NXsSumEmojis = NXsSumEmojis.rename(columns={'index':'sum_emojis'})
NXsSumEmojis


,sum_emojis,NXsSumEmojis
0,0,3
1,37,1
2,300,1
3,58,1
4,1,1


8. NXCs: es la frecuencia correspondiente a cada valor diferente de lacaracterística analizada y que está relacionada con la clase


In [21]:
train

,nature,Class,num_links,sum_emojis
0,bot,1,1,300
1,human,0,0,0
2,bot,1,1,58
3,human,0,0,1
4,bot,1,0,0
5,human,0,0,37
6,bot,1,1,0


In [22]:
NXCsLinks = train[train['Class']==1].groupby(['num_links','Class']).size().reset_index(name='NXCs')
NXCsLinks


,num_links,Class,NXCs
0,0,1,1
1,1,1,3


In [23]:
NXCsEmojis = train[train['Class']==1].groupby(['sum_emojis','Class']).size().reset_index(name='NXCs')
NXCsEmojis


,sum_emojis,Class,NXCs
0,0,1,2
1,58,1,1
2,300,1,1


9. NXNOCs: Es la frecuencia correspondiente a cada valor diferente de la característica analizada y que está relacionado con la no clase



In [24]:
train

,nature,Class,num_links,sum_emojis
0,bot,1,1,300
1,human,0,0,0
2,bot,1,1,58
3,human,0,0,1
4,bot,1,0,0
5,human,0,0,37
6,bot,1,1,0


In [25]:
NXNOCsLinks = train[train['Class']==0].groupby(['num_links','Class']).size().reset_index(name='NXNOCs')
NXNOCsLinks


,num_links,Class,NXNOCs
0,0,0,3


In [26]:
NXNOCsEmojis = train[train['Class']==0].groupby(['sum_emojis','Class']).size().reset_index(name='NXNOCs')
NXNOCsEmojis


,sum_emojis,Class,NXNOCs
0,0,0,1
1,1,0,1
2,37,0,1


10. P(C): Es la probabilidad de la clase, también conocida como hipótesis nula


In [28]:
train

,nature,Class,num_links,sum_emojis
0,bot,1,1,300
1,human,0,0,0
2,bot,1,1,58
3,human,0,0,1
4,bot,1,0,0
5,human,0,0,37
6,bot,1,1,0


In [27]:
PC=train[train['Class']==1].shape[0]/train.shape[0]
PC


0.5714285714285714

11. P(NOC): Es la probabilidad de la no clase


In [30]:
PNOC=train[train['Class']==0].shape[0]/train.shape[0]
PNOC

0.42857142857142855

In [33]:
train

,nature,Class,num_links,sum_emojis
0,bot,1,1,300
1,human,0,0,0
2,bot,1,1,58
3,human,0,0,1
4,bot,1,0,0
5,human,0,0,37
6,bot,1,1,0


In [31]:
PCXLinks = NXCsLinks.merge(NXsLinks,on='num_links',how='left')
PCXLinks['P(C|X)'] = PCXLinks['NXCs']/PCXLinks['NXsLinks']
PCXLinks


,num_links,Class,NXCs,NXsLinks,P(C|X)
0,0,1,1,4,0.25
1,1,1,3,3,1.00


In [36]:
PCXSumEmojis = NXCsEmojis.merge(NXsSumEmojis,on='sum_emojis',how='left')
PCXSumEmojis['P(C|X)'] = PCXSumEmojis['NXCs']/PCXSumEmojis['NXsSumEmojis']
PCXSumEmojis

,sum_emojis,Class,NXCs,NXsSumEmojis,P(C|X)
0,0,1,2,3,0.666667
1,58,1,1,1,1.000000
2,300,1,1,1,1.000000


característica num_links


In [38]:
NXCsLinks['P(X|C)']=NXCsLinks['NXCs']/NC
NXCsLinks

,num_links,Class,NXCs,P(X|C)
0,0,1,1,0.25
1,1,1,3,0.75


In [39]:
NXCsEmojis['P(X|C)']=NXCsEmojis['NXCs']/NC
NXCsEmojis

,sum_emojis,Class,NXCs,P(X|C)
0,0,1,2,0.50
1,58,1,1,0.25
2,300,1,1,0.25


14. P(x|NOC) Es la probabilidad e la característica X dada la no Clase
P(x|NOC)=NXNOC/NNOC


In [40]:
NXNOCsLinks['P(X|NOC)']=NXNOCsLinks['NXNOCs']/NNOC
NXNOCsLinks


,num_links,Class,NXNOCs,P(X|NOC)
0,0,0,3,1.0


In [41]:
NXNOCsEmojis['P(X|NOC)'] = NXNOCsEmojis['NXNOCs']/NNOC
NXNOCsEmojis


,sum_emojis,Class,NXNOCs,P(X|NOC)
0,0,0,1,0.333333
1,1,0,1,0.333333
2,37,0,1,0.333333


#15.Score: Es el puntaje asociado a cada valor diferente de cada una de las características

Son los valores del modelo

Score(X)=log((NXCs/Nc)/(NNOC))


In [42]:
train

,nature,Class,num_links,sum_emojis
0,bot,1,1,300
1,human,0,0,0
2,bot,1,1,58
3,human,0,0,1
4,bot,1,0,0
5,human,0,0,37
6,bot,1,1,0


características num_links


In [43]:
NXCsLinks

,num_links,Class,NXCs,P(X|C)
0,0,1,1,0.25
1,1,1,3,0.75


In [44]:
NXNOCsLinks

,num_links,Class,NXNOCs,P(X|NOC)
0,0,0,3,1.0


In [54]:
ScoreLinks = NXCsLinks.merge(NXNOCsLinks,how='outer',on='num_links')
ScoreLinks['Score']= np.log(ScoreLinks['P(X|C)']/ScoreLinks['P(X|NOC)'])
#ScoreLinks = ScoreLinks.dropna()
ScoreLinks = ScoreLinks.fillna(0)
ScoreLinks
#NaN Not as Number


,num_links,Class_x,NXCs,P(X|C),Class_y,NXNOCs,P(X|NOC),Score
0,0,1,1,0.25,0.0,3.0,1.0,-1.386294
1,1,1,3,0.75,0.0,0.0,0.0,0.000000


In [55]:
ScoreEmojis = NXCsEmojis.merge(NXNOCsEmojis,how='outer',on='sum_emojis')
ScoreEmojis['Score']= np.log(ScoreEmojis['P(X|C)']/ScoreEmojis['P(X|NOC)'])
ScoreEmojis = ScoreEmojis.fillna(0)
ScoreEmojis
#NaN Not as Number


,sum_emojis,Class_x,NXCs,P(X|C),Class_y,NXNOCs,P(X|NOC),Score
0,0,1.0,2.0,0.50,0.0,1.0,0.333333,0.405465
1,58,1.0,1.0,0.25,0.0,0.0,0.000000,0.000000
2,300,1.0,1.0,0.25,0.0,0.0,0.000000,0.000000
3,1,0.0,0.0,0.00,0.0,1.0,0.333333,0.000000
4,37,0.0,0.0,0.00,0.0,1.0,0.333333,0.000000
